In [ ]:
#最初のセットアップに使用するので、1回プログラムを実行したら、次回以降は不要です。
!pip install selenium
!pip install webdriver_manager
!pip install pandas
!pip install chardet

In [2]:
# ---------------------------
# Excelの住所データをAmazonのアドレス帳（住所ページ）に自動入力するツール
# ---------------------------


# 必要なモジュールをインポートする
import time
import pandas as pd
import chardet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup

# データの読み込みと準備
file_path = r""

# Shift_JISで試してみる
try:
    data = pd.read_csv(file_path, encoding='shift_jis')
except UnicodeDecodeError:
    # Shift_JISでダメならUTF-8で試してみる
    try:
        data = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        print("UTF-8 でも Shift_JIS でも読み込めません。別のエンコーディングを試してください。")

# NaN値を空の文字列で埋める
data = data.fillna('')

# WebDriverの設定
# Chrome Driverを管理するためのServiceオブジェクトを作成
webdriver_service = Service(ChromeDriverManager().install())
# Chrome Web Driverインスタンスを新規作成
driver = webdriver.Chrome(service=webdriver_service)

# Amazonにログインをする関数
def login_amazon(username, password):
    try:
        # Amazonのウェブサイトを開く
        driver.get('https://www.amazon.co.jp/')

        # ページが読み込まれるのを待つ
        time.sleep(3)

        # WebDriverWaitとexpected_conditionsを用いて、「ログイン」ボタンが表示されるまで待つ
        login_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'nav-link-accountList')))

        # ログインボタンをクリック
        login_button.click()

        # ログインフォームが読み込まれるのを待つ
        time.sleep(3)

        # ユーザー名とパスワードを入力し、フォームを提出
        driver.find_element(By.NAME, 'email').send_keys(username)
        driver.find_element(By.ID, 'continue').click()
        driver.find_element(By.NAME, 'password').send_keys(password)
        driver.find_element(By.ID, 'signInSubmit').click()

        # ログイン後のページが読み込まれるのを待つ
        time.sleep(5)
    except TimeoutException:
        print("ログイン時にタイムアウトが発生しました、この試行をスキップします。")
        time.sleep(10)
        return

# 住所を追加するための関数
def add_address(address):
    try:
        # 住所管理ページを開く
        driver.get('https://www.amazon.co.jp/a/addresses/add?ref=ya_address_book_add_button')
        # ページが読み込まれるのを待つ
        time.sleep(3)

        # 住所の詳細を入力し、フォームを提出
        full_name_field = driver.find_element(By.NAME, 'address-ui-widgets-enterAddressFullName')
        full_name_field.send_keys(address['ShipName'])

        # 電話番号が0で始まらない場合は、先頭に0を追加
        phone_number_field = driver.find_element(By.NAME, 'address-ui-widgets-enterAddressPhoneNumber')
        phone_number = str(address['ShipPhoneNumber'])
        if not phone_number.startswith('0'):
            phone_number = '0' + phone_number
        phone_number_field.send_keys(phone_number)

        # 指定した都道府県に対応する場合、郵便番号に"0"を追加
        postal_code = str(address['ShipZipCode'])
        if address['ShipPrefecture'] in ['北海道', '青森県', '秋田県', '岩手県']:
            postal_code = "0" + postal_code
        postal_code_part_one = postal_code[:3]
        postal_code_part_two = postal_code[3:]
        postal_code_field_one = driver.find_element(By.ID, 'address-ui-widgets-enterAddressPostalCodeOne')
        postal_code_field_two = driver.find_element(By.ID, 'address-ui-widgets-enterAddressPostalCodeTwo')
        postal_code_field_one.send_keys(postal_code_part_one)
        postal_code_field_two.send_keys(postal_code_part_two)
        postal_code_field_two.send_keys(Keys.TAB)  # フォーカスを移動
        time.sleep(1)

        address_line_1_field = driver.find_element(By.NAME, 'address-ui-widgets-enterAddressLine1')
        address_line_1_field.clear()
        driver.execute_script("arguments[0].value = arguments[1];", address_line_1_field, address['ShipCity'])

        address_line_2_field = driver.find_element(By.NAME, 'address-ui-widgets-enterAddressLine2')
        driver.execute_script("arguments[0].value = arguments[1];", address_line_2_field, address['ShipAddress1'])

        address_line_3_field = driver.find_element(By.NAME, 'address-ui-widgets-enterBuildingOrCompanyName')
        address_line_3_field.send_keys(address['ShipAddress2'])

        address_line_3_field.send_keys(Keys.TAB)  # フォーカスを移動
        time.sleep(3)

        wait = WebDriverWait(driver, 10)
        expander_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@data-action="a-expander-toggle"]')))
        expander_element.click()

        # 「置き配を利用しない」を選択
        do_not_use_delivery_option = wait.until(EC.presence_of_element_located((By.XPATH, '//input[@name="preferredDeliveryLocationOTHER" and @value="CUSTOMER_ATTENDED_ONLY"]')))
        do_not_use_delivery_option.click()
        time.sleep(1)

        # 最後のお届け先のボタンをクリック
        toggle_buttons = driver.find_elements(By.CSS_SELECTOR, 'a[data-a-expander-toggle]')  # ボタンの要素を全て取得
        last_toggle_button = toggle_buttons[-1]  # 最後の要素を取得
        last_toggle_button.click()  # ボタンをクリック
        time.sleep(1)
        # テキストエリアに直接アクセスして値を設定
        textareas = driver.find_elements(By.CSS_SELECTOR, 'textarea[name="freeTextInstruction"]')
        textarea = textareas[-1]
        #お届け先の目印、アクセス方法に任意の文章を入力する
        note = str('お届け先の目印、アクセス方法に任意の文章を入力する')
        textarea.send_keys(note)
        
        # 住所を追加するボタンを押す
        submit_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//input[@aria-labelledby="address-ui-widgets-form-submit-button-announce"]')))
        submit_button.click()
        time.sleep(5)

        # 住所追加後に警告メッセージが表示されているか確認
        try:
            warning_message = driver.find_element(By.XPATH, "//div[@class='a-box a-alert a-alert-warning a-spacing-none']")
            if warning_message:
                print(f"{address['OrderId']} の {address['ShipName']} の住所が追加されませんでした。")
                return  # 関数から抜け出す
        except NoSuchElementException:
            pass  # 警告メッセージが存在しない場合は何もしない

        # 正常に追加されたメッセージを表示
        print(f"{address['OrderId']} の {address['ShipName']} の住所が追加されました。")

    except TimeoutException:
        print("タイムアウトが発生しました。再試行してください。")
        time.sleep(10)

# Amazonにログイン
login_amazon('email_address', 'amazon_password')

# CSVから読み込んだ住所データを追加
for index, row in data.iterrows():
    # 現在の行の住所を追加
    add_address(row)
    time.sleep(3)  # 追加が完了するまで待つ

# ブラウザウィンドウを閉じる
driver.quit()

10001063 の 中林 圭子 の住所が追加されました。


In [5]:
# 必要なモジュールをインポートする
import time
import pandas as pd
import chardet
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException  


# データの読み込みと準備
file_path = r"C:\Users\Administrator\Documents\address\address.csv"

# Shift_JISで試してみる
try:
    data = pd.read_csv(file_path, encoding='shift_jis')
except UnicodeDecodeError:
    # Shift_JISでダメならUTF-8で試してみる
    try:
        data = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        print("UTF-8 でも Shift_JIS でも読み込めません。別のエンコーディングを試してください。")

# NaN値を空の文字列で埋める
data = data.fillna('')

# WebDriverの設定
# Chrome Driverを管理するためのServiceオブジェクトを作成
webdriver_service = Service(ChromeDriverManager().install())
# Chrome Web Driverインスタンスを新規作成
driver = webdriver.Chrome(service=webdriver_service)

# Amazonにログインをする関数
def login_amazon(username, password):
    try:
        # Amazonのウェブサイトを開く
        driver.get('https://www.amazon.co.jp/')
        # ページが読み込まれるのを待つ
        time.sleep(3)
        # WebDriverWaitとexpected_conditionsを用いて、「ログイン」ボタンが表示されるまで待つ
        login_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'nav-link-accountList')))
        # ログインボタンをクリック
        login_button.click()
        # ログインフォームが読み込まれるのを待つ
        time.sleep(3)
        # ユーザー名とパスワードを入力し、フォームを提出
        driver.find_element(By.NAME, 'email').send_keys(username)
        driver.find_element(By.ID, 'continue').click()
        driver.find_element(By.NAME, 'password').send_keys(password)
        driver.find_element(By.ID, 'signInSubmit').click()
        # ログイン後のページが読み込まれるのを待つ
        time.sleep(10)
    except TimeoutException:
        print("ログイン時にタイムアウトが発生しました、この試行をスキップします。")
        time.sleep(60)
        return

# Amazonのアドレスを削除する関数
def delete_address(driver, phone_number, zip_code):
    wait = WebDriverWait(driver, 10)
    driver.get('https://www.amazon.co.jp/a/addresses/')
    time.sleep(2)
    driver.refresh()  # ページをリフレッシュ
    time.sleep(2)
    # 住所要素を検索する
    address_elements = driver.find_elements(By.XPATH, "//div[@class='a-box-inner a-padding-none']//div[@class='a-section address-section-no-default']")
    if not address_elements:
        return
    # 削除する住所のインデックスを格納するリスト
    addresses_to_delete = []
    # 各住所要素をループ処理する
    for index, address_element in enumerate(address_elements):
        try:
            # 郵便番号と電話番号の要素を検索する
            zip_element = address_element.find_element(By.XPATH, f".//span[contains(text(), '{zip_code}')]")
            phone_element = address_element.find_element(By.XPATH, f".//span[contains(text(), '{phone_number}')]")
            time.sleep(2)
            # 郵便番号と電話番号が一致する場合に削除処理を行う
            if zip_element and phone_element:
                addresses_to_delete.append(index)
        except NoSuchElementException:
            continue
    # addresses_to_deleteリストの住所を削除する
    for index in addresses_to_delete:
        # address_elementsを再取得する
        address_elements = driver.find_elements(By.XPATH, "//div[@class='a-box-inner a-padding-none']//div[@class='a-section address-section-no-default']")
        if not address_elements:
            continue
        address_element = address_elements[index]
        parent_container = address_element.find_element(By.XPATH, "./ancestor::div[contains(@class, 'a-column') and contains(@class, 'a-span4') and contains(@class, 'a-spacing-none') and contains(@class, 'a-spacing-top-mini') and contains(@class, 'address-column')]")
        delete_button = parent_container.find_element(By.XPATH, ".//a[@class='a-link-normal delete-link']")
        delete_button.click()
        time.sleep(2)
        # 削除を確認するポップアップが表示されるのを待つ
        confirm_popup = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='a-popover-wrapper' and contains(.//h4, '削除を確認')]")))
        # ポップアップ内の「はい」ボタンをクリック
        confirm_button = confirm_popup.find_element(By.XPATH, ".//input[@type='submit']")
        confirm_button.click()
        # 削除が完了するまで待つ
        wait.until(EC.staleness_of(confirm_popup))
        time.sleep(2)
    return len(addresses_to_delete) > 0

def modify_data(row):
    phone_number = str(row['ShipPhoneNumber'])
    zip_code = str(row['ShipZipCode'])

    # 電話番号が '0' で始まるようにする
    if not phone_number.startswith('0'):
        phone_number = '0' + phone_number

    # 郵便番号が '0' で始まらない場合、かつ都道府県が以下のいずれかである場合は、郵便番号の先頭に '0' を追加する
    if row['ShipPrefecture'] in ['北海道', '青森県', '秋田県', '岩手県'] and not zip_code.startswith('0'):
        zip_code = '0' + zip_code

    # 郵便番号の3文字目の後に '-' を挿入する
    if len(zip_code) >= 3:
        zip_code = zip_code[:3] + '-' + zip_code[3:]
    return phone_number, zip_code   

# Amazonにログイン
login_amazon('email_address', 'amazon_password')

for index, row in data.iterrows():
    phone_number, zip_code = modify_data(row)
    success = delete_address(driver, phone_number, zip_code)
    if success:
        print(f" {row['OrderId']} の {row['ShipName']} の住所が削除されました。")
    else:
        print(f" {row['OrderId']} の {row['ShipName']} の住所が削除されませんでした。")

# WebDriverを終了する
driver.quit()

# ファイルを削除する
os.remove(file_path)

 10001063 の 中林 圭子 の住所が削除されました。
